In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flowers-recognition


In [2]:
import os

images_directory = os.path.join("/kaggle/input/flowers-recognition/flowers", "flower_images")
print("Images directory:", images_directory)

Images directory: /kaggle/input/flowers-recognition/flowers/flower_images


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

train_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True )

In [4]:
val_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

In [5]:
target_size = (224, 224)
batch_size = 32

In [6]:
train_data = train_aug.flow_from_directory(
    "/kaggle/input/flowers-recognition/flowers",
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 3457 images belonging to 5 classes.


In [7]:
val_data = val_aug.flow_from_directory(
    "/kaggle/input/flowers-recognition/flowers",
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 860 images belonging to 5 classes.


In [8]:
from tensorflow.keras.applications import ResNet50
resnet_base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
resnet_base.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [10]:
model = Sequential()

In [11]:
model.add(resnet_base)
model.add(layers.GlobalAveragePooling2D())

In [12]:
model.add(layers.Dense(256))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.4))

In [13]:
model.add(layers.Dense(128))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.3))

In [14]:
model.add(layers.Dense(64))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

In [15]:
model.add(layers.Dense(5, activation='softmax'))

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [20]:
model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 70s 646ms/step - accuracy: 0.6477 - loss: 0.9242 - val_accuracy: 0.8581 - val_loss: 0.4113
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 48s 437ms/step - accuracy: 0.8499 - loss: 0.4474 - val_accuracy: 0.8907 - val_loss: 0.2977
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 48s 440ms/step - accuracy: 0.8729 - loss: 0.3851 - val_accuracy: 0.8930 - val_loss: 0.2907
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 48s 437ms/step - accuracy: 0.8729 - loss: 0.3531 - val_accuracy: 0.9128 - val_loss: 0.2851
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 48s 440ms/step - accuracy: 0.9020 - loss: 0.2909 - val_accuracy: 0.8988 - val_loss: 0.2683
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 47s 433ms/step - accuracy: 0.8971 - loss: 0.3013 - val_accuracy: 0.8977 - val_loss: 0.2852
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 48s 439ms/step - accuracy: 0.9232 - loss: 0.2435 - val_accuracy: 0.9000 - val_loss: 0.2726
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 49s 447ms/step - accuracy: 0.9217 - loss: 0